In [1]:
from nms_cupy import cupy_call_nms_kernel,_load_kernel,_nms_gpu_code 
import numpy as np
import cupy as cp
from nms_numba_cpu import nms_cpu
from nms_numba import nms_kernel,numba_call_nms_kernel
from numba import cuda
thresh=0.7

1.ubuntu 18    
2.python 3.7    
3.cuda   9.2    
4.gpu  2080ti    

# 1.numba

In [13]:
bbox=np.load("bbox.npy")
n_bbox = bbox.shape[0]
threads_per_block = 64
col_blocks = np.ceil(n_bbox / threads_per_block).astype(np.int32)
blocks = (col_blocks, col_blocks, 1)
threads = (threads_per_block, 1, 1)

mask_dev = cuda.device_array((n_bbox * col_blocks,), dtype=np.uint64)
bbox=bbox.reshape(-1)
bbox =cuda.to_device(bbox)

%timeit nms_kernel[blocks, threads](n_bbox, bbox, mask_dev)


15.5 ms ± 229 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
bbox=np.load("bbox.npy")
numba_call_nms_kernel(bbox,0.7)

(array([ 0,  9, 41, ...,  0,  0,  0], dtype=int32), 1736)

# 2.cupy #

In [12]:
bbox=np.load("bbox.npy")
bbox=cp.asarray(bbox)
threads_per_block = 64
col_blocks = np.ceil(n_bbox / threads_per_block).astype(np.int32)
blocks = (col_blocks, col_blocks, 1)
threads = (threads_per_block, 1, 1)

mask_dev = cp.zeros((n_bbox * col_blocks,), dtype=np.uint64)
bbox = cp.ascontiguousarray(bbox, dtype=np.float32)
kern = _load_kernel('nms_kernel', _nms_gpu_code)
%timeit kern(blocks, threads, args=(cp.int32(n_bbox), cp.float32(thresh),bbox, mask_dev))

1.33 ms ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
bbox=np.load("bbox.npy")
bbox=cp.asarray(bbox)
%timeit cupy_call_nms_kernel(bbox,0.7)
# print(n_selec.get())

3.37 ms ± 5.09 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 3.numba_cpu #

In [11]:
bbox=np.load("bbox.npy")
%timeit keep=nms_cpu(bbox,0.7)
# print(keep.shape)

33.1 ms ± 127 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# 4.pybind11 cuda

In [9]:
import nms 
import numpy as np

bbox=np.load("bbox.npy")
n_bbox = bbox.shape[0]

keep_out=np.zeros(bbox.shape[0],dtype=np.int32)
%timeit n=nms.nms_cuda(keep_out,bbox)
print(keep_out)
print(n)

5.76 ms ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
[ 0  9 41 ...  0  0  0]
1713
